In [1]:
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [2]:
#!/usr/bin/env python3
#coding=utf-8
import time
from Arm_Lib import Arm_Device
import  serial
import threading

ser = serial.Serial('/dev/ttyACM0',9600,timeout=1)
Arm = Arm_Device()
time.sleep(.1)

In [3]:
from Arm_Lib import Arm_Device

Arm = Arm_Device()


In [4]:
import traitlets
import ipywidgets.widgets as widgets
import time
import threading
import inspect
import ctypes

origin_widget = widgets.Image(format='jpeg', width=320, height=240)
mask_widget = widgets.Image(format='jpeg',width=320, height=240)
result_widget = widgets.Image(format='jpeg',width=320, height=240)

image_container = widgets.HBox([origin_widget, mask_widget, result_widget])
# image_container = widgets.Image(format='jpeg', width=600, height=500)
display(image_container)

In [5]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [6]:
def get_color(img):
    H = []
    color_name={}
    img = cv2.resize(img, (640, 480), )
    HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    cv2.rectangle(img, (280, 180), (360, 260), (0, 255, 0), 2)
    for i in range(280, 360):
        for j in range(180, 260): H.append(HSV[j, i][0])
    H_min = min(H);H_max = max(H)
#     print(H_min,H_max)
    if H_min >= 0 and H_max <= 6 or H_min >= 156 and H_max <= 180: color_name['name'] = 'red'
    elif H_min >= 21 and H_max <= 34: color_name['name'] = 'yellow'
    elif H_min >= 52 and H_max <= 84: color_name['name'] = 'green'
    elif H_min >= 96 and H_max <= 124: color_name['name'] = 'blue'
    return img, color_name

In [7]:
# Define variable parameters at different locations
# 定义不同位置的变量参数
look_at = [90, 130, 0, 0, 90, 30]
p_top = [90, 80, 50, 50, 270]

p_Yellow = [65, 22, 64, 56, 270]
p_Red = [118, 19, 66, 56, 270]

p_Green = [136, 66, 20, 29, 270]
p_Blue = [44, 66, 20, 28, 270]

p_gray = [90, 48, 35, 30, 270]



In [8]:
# Define control DOFBOT function, control No.1-No.6 servo，p=[S1,S2,S3,S4,S5,S6]
# 定义移动机械臂函数,同时控制1-6号舵机运动，p=[S1,S2,S3,S4,S5,S6]
def arm_move_6(p, s_time = 500):
    for i in range(6):
        id = i + 1
        Arm.Arm_serial_servo_write(id, p[i], s_time)
        time.sleep(.01)
    time.sleep(s_time/1000)
    
# Define control DOFBOT function, control No.1-No.5 servo，p=[S1,S2,S3,S4,S5]
# 定义移动机械臂函数,同时控制1-5号舵机运动，p=[S1,S2,S3,S4,S5]
def arm_move(p, s_time = 500):
    for i in range(5):
        id = i + 1
        if id == 5:
            time.sleep(.1)
            Arm.Arm_serial_servo_write(id, p[i], int(s_time*1.2))
        elif id == 1 :
            Arm.Arm_serial_servo_write(id, p[i], int(3*s_time/4))
        else:
            Arm.Arm_serial_servo_write(id, p[i], int(s_time))
        time.sleep(.01)
    time.sleep(s_time/1000)
    
# enable=1：clip，=0：release
def arm_clamp_block(enable):
    if enable == 0:
        Arm.Arm_serial_servo_write(6, 60, 400)
    else:
        Arm.Arm_serial_servo_write(6, 145, 400)
    time.sleep(.5)

In [9]:
arm_move_6(look_at, 1000)
time.sleep(1)

In [10]:
global g_state_arm
g_state_arm = 0

def ctrl_arm_move(index):
    arm_clamp_block(0)
    if index == 1:
        print("Yellow")

    elif index == 2:
        print("Red")

    elif index == 3:
        print("Green")

    elif index == 4:
        print("Blue")

    
    global g_state_arm
    g_state_arm = 0


In [11]:
# #Define the function corresponding to each number
# 数字功能定义
def number_action(index):
    if index == 1:
        # Grab yellow block
        # 抓取黄色的积木块

        arm_clamp_block(0)

    elif index == 2:
        # Grab red block
        # 抓取红色的积木块

        arm_clamp_block(0)

    elif index == 3:
        # Grab green block
        # 抓取绿色的积木块

        arm_clamp_block(0)

    elif index == 4:
        # Grab blue block
        # 抓取蓝色的积木块

        arm_clamp_block(0)

    
def put_down_block():

    arm_move_6(look_at, 1000)

    

In [12]:
import cv2
import numpy as np
def out_xy(img):
    x,y=0,0
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    kernel = np.ones((1, 5), np.uint8)
    t, binary = cv2.threshold(gray, 90, 255, cv2.THRESH_BINARY_INV)

    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, anchor=(2, 0), iterations=5)

    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    cv2.drawContours(img, contours, -1, (0, 0, 255), 5)
    if(len(contours)!=0):
        x,y,w,h = cv2.boundingRect(contours[0])
    return x,y

In [13]:
def start_move_arm(index):
    global g_state_arm
    if g_state_arm == 0:
        closeTid = threading.Thread(target = ctrl_arm_move, args = [index])
        closeTid.setDaemon(True)
        closeTid.start()
        
        g_state_arm = 1

In [14]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

def calculate_loc(x,y):
    data = pd.read_csv('servo.csv')
    X = data.iloc[:,0:2]
    Y = data.iloc[:,2:]
    neigh = KNeighborsClassifier(n_neighbors=3, weights='distance')
    neigh.fit(X, Y)
    a = neigh.predict([[x, y]])
    final=[]
    final = list(a[0])
    final.append(270)
    final.append(0)
    arm_move_6(final, 1000)
    ser.write(b'o')

    

    time.sleep(2)
    return 0
    

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-c1w2v_iy because the default path (/home/jetson/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [15]:
import time
import cv2
def stable_xy(x,y):
    if(x+y!=0):
        cap = cv2.VideoCapture(0)
        cap.set(3, 640)
        cap.set(4, 480)
        cap.set(5, 30)  
        cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
        time.sleep(1)
        a=[0,0]
        b=[0,0]
        while(1):
            ret, frame = cap.read()
            b[0:2]=out_xy(frame)
            res=abs(a[0]-b[0])+abs(a[1]-b[1])
            if(res<10):
                break                
            a[0:2]=b[0:2]
        x=a[0]
        y=a[1]
        return x,y


In [16]:
# Grab a block from the gray block and put it on the red block.
# 从灰色积木块位置抓取一块积木放到红色积木块的位置上。
def arm_hard():
    

    arm_move(p_top, 1000)
    arm_move(p_Red, 1000)
    arm_clamp_block(0)

    
    arm_move(look_at, 1100)

    time.sleep(1)


In [17]:
# Grab a block from the gray block and put it on the yellow block.
# 从灰色积木块位置抓取一块积木放到黄色积木块的位置上。
def arm_soft():
    
    arm_move(p_top, 1000)
    arm_move(p_Yellow, 1000)
    arm_clamp_block(0)


    arm_move(look_at, 1000)

    time.sleep(1)

In [18]:
import cv2
import numpy as np
import ipywidgets.widgets as widgets


cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
cap.set(5, 30)  
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
# image.set(cv2.CAP_PROP_BRIGHTNESS, 40) 
# image.set(cv2.CAP_PROP_CONTRAST, 50)   
# image.set(cv2.CAP_PROP_EXPOSURE, 156)  

# Red is selected by default, and the program will automatically switch the color according to the color detected in the box
# 默认选择红色的,程序会自动根据方框中检测到的颜色切换颜色
# Red value
# 红色区间

# green value
# 绿色区间
# color_lower = np.array([35, 43, 46])
# color_upper = np.array([77, 255, 255])

# blue value
# #蓝色区间
# color_lower=np.array([100, 43, 46])
# color_upper = np.array([124, 255, 255])

# yellow value
# #黄色区间
# color_lower = np.array([26, 43, 46])
# color_upper = np.array([34, 255, 255])

# orange value
# #橙色区间
# color_lower = np.array([11, 43, 46])
# color_upper = np.array([25, 255, 255])


def Color_Recongnize():
    global zuobiao
    zuobiao=np.zeros([10,2])
    Arm.Arm_Buzzer_On(1)
    s_time = 300
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 10, s_time)
    time.sleep(s_time/1000)
    Arm.Arm_serial_servo_write(4, 0, s_time)
    time.sleep(s_time/1000)
    count=0
    while(1):
        chary=0
        ret, frame = cap.read()
        frame = cv2.resize(frame, (640, 480), )

        #cv2.imshow('Capture', frame)

        # change to hsv model
        x,y=out_xy(frame)
        if(x+y!=0):
            if(count<=9):
                zuobiao[count,0]=x
                zuobiao[count,1]=y
                count+=1
            elif(count==10):
                zuobiao=zuobiao[np.lexsort(zuobiao.T)]
                x=zuobiao[4,0]
                y=zuobiao[4,1]
                print(x,y)
                calculate_loc(x,y)
                flag = ser.inWaiting()
                count=0
                while(chary==0):
                    if(flag!=0):
                        
                        if(b'a'==ser.read()):
                            ser.flush()
                            arm_clamp_block(1)
                            while True:
                                flag = ser.inWaiting()
                                if (flag!=0):
                        
                        #S数据转换
                                    a = ser.read(3)
                                    print(a,end='\n')
                                    b = bytes.decode(a)
                                    value = int(b)
                                    print(value)
                        
                        #软硬判断
                                    if(value>500):
                                        arm_hard()
                                        chary=1
                                        break
                                    else:
                                        arm_soft()
                                        chary=1
                                        break

                        
        time.sleep(0.01)



    cap.release()
    #cv2.destroyAllWindows()

In [19]:
thread1 = threading.Thread(target=Color_Recongnize)
thread1.setDaemon(True)
thread1.start()

184.0 177.0
b'516'
516
185.0 326.0


In [ ]:
#End process, only need to execute this code at the end
# #结束进程，只有在结束时才需要执行此段代码
try:
    while True:
        time.sleep(.000001)
except KeyboardInterrupt:
    print(" Program closed! ")
    pass
stop_thread(thread1)

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-18-58e42b32c852>", line 58, in Color_Recongnize
    frame = cv2.resize(frame, (640, 480), )
cv2.error: OpenCV(4.1.1) /home/nvidia/host/build_opencv/nv_opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'




In [ ]:
cap.release()            